In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import ast  # For safe evaluation of Python-like strings
from transformers import BertTokenizer
import pickle as pkl
from dataset import RedlistDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import AdamW
from model import RedlistPredictor
from tqdm import tqdm
from transformers import get_scheduler
import os
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [3]:
# Load the provided CSV content into a pandas DataFrame
csv_path = "data/DD_true.csv"
df = pd.read_csv(csv_path)
# tools.display_dataframe_to_user(name="Data Deficient Species Data", dataframe=df)

In [4]:
df = df.drop(columns=['index', 'IUCN Red List (2024) Category', 'measures', 'trend_justification'])

In [5]:
df

,scientificName,redlistCategory,range,population,habitats,threats,use_trade,rationale,taxonomic_notes
0,Berylmys mackenziei,LC,This poorly-known species has been reported fr...,It appears to be a locally abundant species in...,It is a nocturnal and fossorial species occurr...,The threats to this poorly-known species in So...,NaN,This species is listed as Least Concern as it ...,The single specimen from Sichuan is larger and...
1,Berylmys manipulus,LC,This species is present in India in Assam (Gol...,"In central Myanmar, it is fairly common in for...",It has been recorded from evergreen rainforest...,"The major threat is likely to be forest loss, ...",It is hunted for food in India.,This species is listed as Least Concern as it ...,Thomas in 1916 described the Manipuri rat from...
2,Cabassous centralis,LC,"<span style=""font-style: italic;"">Cabassous ce...","<span style=""font-style: italic;"">Cabassous ce...","<span style=""font-style: italic;"">Cabassous ce...",The exact threats to this species are not know...,The species is not utilized.,"<p><span lang=""EN-US""><em>Cabassous centralis<...","The wide range of <span style=""font-style: ita..."
3,Crocidura musseri,VU,This species is endemic to the island of Sulaw...,The abundance and population size of this spec...,It is found in mossy montane forests on Gunung...,The specific threats to this species are not k...,NaN,This species is known from few localities. Bas...,NaN
4,Aboma etheostoma,LC,This species is distributed in the Tropical Ea...,This species is common and can be locally abun...,This demersal species primarily inhabits shall...,This species is locally impacted by loss of ha...,This species is not utilized.,This estuarine species is widely distributed. ...,NaN
...,...,...,...,...,...,...,...,...,...
78,Prospero rhadamanthi,CR,"This species is endemic to Crete, Greece, wher...",<p>The number of mature individuals is estimat...,"The species is found in stony ground, openings...",The main threat to the species is trampling fr...,NaN,This species is only known from the Petres Gor...,<em>Prospero rhadamanthi </em>was described by...
79,Prunus herthae,VU,"Especie nativa de Ecuador y Perú, concentradas...",No se dispone de datos poblacionales para la e...,Este&#160;es un árbol&#160;de hasta 20 m de al...,"En su rango geográfico, la deforestación, conc...",No se han reportado usos para la especie.,<em>Prunus herthae</em> es un árbol distribuid...,NaN
80,Tamarix minoa,VU,<p>This species is endemic to the Georgioupoli...,<p>The species forms a dense population. The p...,"<p> </p><p>Shrubs or low shrubby trees, up to...",<p>There seem to be no apparent current threat...,NaN,"<p>This species is endemic to Greece, where it...","In Crete, this species was previously consider..."
81,Tulipa hageri,EN,<em>Tulipa hageri</em> is an endemic species r...,According to Pipinis <em>et al</em>. (2023) an...,Dimopoulos <em>et al</em>. (2022) describe the...,"According to Pipinis <em>et al</em>. (2023), <...",The species is widely cultivated for ornamenta...,The distribution of <em>Tulipa hageri </em>enc...,"WCSP (2017, citing Tutin <em>et al.</em> 1980,..."


In [5]:
with open('results.pkl', 'rb') as f:
    test_res = pkl.load(f)
    test_preds = pkl.load(f)
    test_true_labels = pkl.load(f)
    deficient_predictions = pkl.load(f)
    deficient_confidences = pkl.load(f)
    deficient_all_probs = pkl.load(f)

In [ ]:
with open('results.pkl', 'rb') as f:
    _ = pkl.load(f)
    _ = pkl.load(f)
    _ = pkl.load(f)
    deficient_predictions = pkl.load(f)
    deficient_confidences = pkl.load(f)
    deficient_all_probs = pkl.load(f)

In [3]:
with open("data/data_splits.pkl", "rb") as f:
    data = pkl.load(f)

df_train = data["train"]
df_test = data["test"]
df_deficient = data["deficient"]

In [16]:
df_all = pd.concat([df_train, df_test, df_deficient], axis=0)

In [13]:
pd.set_option('display.max_colwidth', None)  # Set to None to display the full content of each cell
for index, row in df_deficient.iterrows():
    print(row['populationTrend'])
    print()

1

0

1

0

0

0

0

0

0

1

0

0

0

0

1

0

0

0

0

1

1

0

0

0

0

1

0

0

0

1

0

1

0

0

1

0

1

0

1

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

1

0

0

0

2

0

0

0

0

0

2

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

1

0

0

0

0

0

0

2

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

1

2

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

1

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

0

In [7]:
df_deficient['rationale']

3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [7]:
scientificName_to_probs = dict(zip(df_deficient['scientificName'], deficient_all_probs))

In [8]:
df_first2 = df.iloc[:, :2].copy()
df_rest = df.iloc[:, 2:].copy()

In [9]:
# Define the labels corresponding to the probabilities

# label_to_category = {
#     0: 'Least Concern',
#     1: 'Near Threatened',
#     2: 'Vulnerable',
#     3: 'Endangered',
#     4: 'Critically Endangered',
#     5: 'Extinct'
# }
label_to_category = {
    0: 'LC',
    1: 'NT',
    2: 'VU',
    3: 'EN',
    4: 'CR',
    5: 'EX'
}

# Function to extract top labels and probabilities
def extract_top_probs(scientific_name):
    probs = scientificName_to_probs.get(scientific_name, None)
    if probs is not None:
        sorted_indices = probs.argsort()[::-1]  # Sort indices in descending order of probabilities
        top_labels = [label_to_category[i] for i in sorted_indices]
        top_probs = [probs[i] for i in sorted_indices]
        return top_labels, top_probs
    else:
        return [None] * 6, [None] * 6


for i in range(6):
    df_first2[f'top {i + 1} label'] = df_first2['scientificName'].map(lambda x: extract_top_probs(x)[0][i])
    df_first2[f'top {i + 1} prob'] = df_first2['scientificName'].map(lambda x: extract_top_probs(x)[1][i])

# Reorder the columns as specified
# top_labels = ['top 1 label', 'top 2 label', 'top 3 label', 'top 4 label', 'top 5 label', 'top 6 label']
# top_probs = ['top 1 prob', 'top 2 prob', 'top 3 prob', 'top 4 prob', 'top 5 prob', 'top 6 prob']
# reordered_columns = [col for pair in zip(top_labels, top_probs) for col in pair]

# Insert the reordered columns starting from the 3rd column
# df = pd.concat([df.iloc[:, :2], df[reordered_columns], df.drop(columns=reordered_columns, errors='ignore')], axis=1)

In [10]:
df = pd.concat([df_first2, df_rest], axis=1)

In [11]:
df = df[df['top 1 label'].notna()]

In [13]:
fractions = {}
for i in range(1, 7):
    match_count = (df['redlistCategory'] == df[f'top {i} label']).sum()
    fractions[f'top {i} label'] = match_count / len(df)

for i in range(1, 7):
    # cur = fractions[f'top {i} label']
    cummulative = sum(fractions[f'top {j} label'] for j in range(1, i + 1))
    print(f"Matching top {i} labels: {cummulative:.2%}")

Matching top 1 labels: 79.55%
Matching top 2 labels: 93.18%
Matching top 3 labels: 95.45%
Matching top 4 labels: 100.00%
Matching top 5 labels: 100.00%
Matching top 6 labels: 100.00%


In [ ]:
# df.to_csv("DD_species.csv", index=False)